# Make a dump from a sqllite database as a backup
- https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.iterdump

In [1]:
import sqlite3
from pathlib import Path

In [2]:
DATA_PATH = Path().resolve().parent.parent/"data"
# print(DATA_PATH)

## Create dummy database

### Load raw dummy data

In [3]:
raw_file = DATA_PATH/"datasets"/"simplemaps_worldcities.csv"
print(raw_file.is_file())  # check if exists

# check first lines
with open(raw_file, "r") as rfile:
    lines = rfile.readlines()

print(len(lines))
lines[:5]

True
26570


['"city","city_ascii","lat","lng","country","iso2","iso3","admin_name","capital","population","id"\n',
 '"Tokyo","Tokyo","35.6897","139.6922","Japan","JP","JPN","Tōkyō","primary","37977000","1392685764"\n',
 '"Jakarta","Jakarta","-6.2146","106.8451","Indonesia","ID","IDN","Jakarta","primary","34540000","1360771077"\n',
 '"Delhi","Delhi","28.6600","77.2300","India","IN","IND","Delhi","admin","29617000","1356872604"\n',
 '"Mumbai","Mumbai","18.9667","72.8333","India","IN","IND","Mahārāshtra","admin","23355000","1356226629"\n']

In [4]:
# create db in memory
try:
    conn = sqlite3.connect(":memory:")
except sqlite3.Error as e:
    print(e)

# Table Definition
create_table = """
    CREATE TABLE IF NOT EXISTS simplemaps_worldcities(
        city VARCHAR(255) NOT NULL,
        city_ascii VARCHAR(255) NOT NULL,
        lat DOUBLE NOT NULL,
        lng DOUBLE NOT NULL,
        country VARCHAR(255) NOT NULL,
        iso2 VARCHAR(255) NOT NULL,
        iso3 VARCHAR(255) NOT NULL,
        admin_name VARCHAR(255) NOT NULL,
        capital VARCHAR(255) NOT NULL,
        population INTEGER NOT NULL,
        id INTEGER PRIMARY KEY
    );
"""

cursor = conn.cursor()
cursor.execute(create_table)

# Insert values
# insert_values = """
#     INSERT INTO simplemaps_worldcities (
#         city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
#     )
#     VALUES
#     ("Tokyo","Tokyo",35.6897,139.6922,"Japan","JP","JPN","Tōkyō","primary",37977000,1392685764);
# """
# cursor.execute(insert_values)
for line in lines[1:]:
    insert_values = f"""
        INSERT INTO simplemaps_worldcities (
            city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
        )
        VALUES
        ({line});
    """
    cursor.execute(insert_values)

# cursor.execute("""SELECT "name", "type", "sql" FROM sqlite_master;""")
# cursor.fetchall()

with open(DATA_PATH/"dump.sql", "w") as f:
    for line in conn.iterdump():
        f.write(f"{line}\n")

In [5]:
# create db in memory
try:
    conn = sqlite3.connect(":memory:")
except sqlite3.Error as e:
    print(e)

with open(DATA_PATH/"dump.sql", "r") as sql_file:
    sql_script = sql_file.read()

cursor = conn.cursor()
cursor.executescript(sql_script)

cursor.execute("""SELECT "name", "type", "sql" FROM sqlite_master;""")
cursor.fetchall()

[('simplemaps_worldcities',
  'table',
  'CREATE TABLE simplemaps_worldcities(\n        city VARCHAR(255) NOT NULL,\n        city_ascii VARCHAR(255) NOT NULL,\n        lat DOUBLE NOT NULL,\n        lng DOUBLE NOT NULL,\n        country VARCHAR(255) NOT NULL,\n        iso2 VARCHAR(255) NOT NULL,\n        iso3 VARCHAR(255) NOT NULL,\n        admin_name VARCHAR(255) NOT NULL,\n        capital VARCHAR(255) NOT NULL,\n        population INTEGER NOT NULL,\n        id INTEGER PRIMARY KEY\n    )')]